# Importing libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from scipy.sparse import csr_matrix
import pickle
# import cv2 as cv
# import time

# Defining paths

In [2]:
cur_dir = 'C:/Users/cortiz/Projects/inscopix_project_cajal/data/20211204/mouse_02_IL'
path_table = os.path.join(cur_dir, 'behavior', 'table_cut.csv')
path_mat_spikes = os.path.join(cur_dir, 'spikes', 'finalSpikesMat.mat')

# Parameters

In [3]:
framerate_inscopix = 20
framerate_behavior = 15
frame_count = np.array([4015, 3709, 3872, 3715, 3737, 3752, 4344, 3698, 4628, 3801, 3770, 3774])
discard_sessions = np.array([8,9])
dict_env = {'t_maze': [0, 3, 6],
            'y_maze': [1, 4, 7, 8],
            'open_field': [2, 5, 9]}

# Loading files

In [5]:
path_pickle = os.path.join(cur_dir, 'pickle')
with open(os.path.join(path_pickle, 'meta_spikes.pck'), 'rb') as f:
    meta_spikes = pickle.load(f)
with open(os.path.join(path_pickle, 'df_dlc.pck'), 'rb') as f:
    df_dlc = pickle.load(f)
with open(os.path.join(path_pickle, 'mat_spikes.pck'), 'rb') as f:
    mat_spikes = pickle.load(f)
with open(os.path.join(path_pickle, 'df_frames.pck'), 'rb') as f:
    df_frames = pickle.load(f)

### Checking offset between recordings

# Plotting spatial activity

In [6]:
sorted_cells_decreasing = np.argsort(-np.sum(mat_spikes, axis = 0))

ncol = 4
nrow = int(np.ceil(len(df_frames)/ncol))

for cur_cell in sorted_cells_decreasing[:15]:
    plt.figure(figsize = (5*ncol, 5*nrow))
    spike_train = np.where(mat_spikes[:, cur_cell])[0]

    for i,r in df_frames.iterrows():
        cur_session = r['session']
        plt.subplot(nrow, ncol, i+1)
        sub_spikes = meta_spikes.query('session == @cur_session')
        cur_spikes = spike_train[[s in sub_spikes.index for s in spike_train]]
        plt.title(cur_session)
        plt.plot(sub_spikes['x'], sub_spikes['y'], c = 'lightgray', alpha = 0.5, linewidth = 0.5)
        plt.scatter(meta_spikes['x'][cur_spikes], meta_spikes['y'][cur_spikes])
        plt.gca().set_aspect('equal')
        
    plt.savefig(cur_dir + '/plots/activity_overview_cell_' + str(cur_cell).zfill(3) + '.png', dpi = 300)
    plt.close()

In [ ]:
sorted_cells_decreasing = np.argsort(-np.sum(mat_spikes, axis = 0))

ncol = 3
nrow = 1

for cur_cell in sorted_cells_decreasing[:15]:
    
    plt.figure(figsize = (5*ncol, 5*nrow))
    spike_train = np.where(mat_spikes[:, cur_cell])[0]
    cnt_plt = 0
    
    for k, selected_sessions in dict_env.items():
        
        cnt_plt += 1
        plt.subplot(nrow, ncol, cnt_plt)
        plt.title(cur_session)
        plt.gca().set_aspect('equal')
        
        lname = list()
        for idx in selected_sessions:
            lname.append(df_frames.loc[idx, 'session'])

        for i,r in df_frames.iterrows():
            if r['session'] not in lname:
                continue
            
            cur_session = r['session']
            sub_spikes = meta_spikes.query('session == @cur_session')
            cur_spikes = spike_train[[s in sub_spikes.index for s in spike_train]]
            plt.plot(sub_spikes['x'], sub_spikes['y'], c = 'lightgray', alpha = 0.5, linewidth = 0.5)
            plt.scatter(meta_spikes['x'][cur_spikes], meta_spikes['y'][cur_spikes])
        
    plt.savefig(cur_dir + '/plots/activity_env_merged_cell_' + str(cur_cell).zfill(3) + '.png', dpi = 300)
    plt.close()